In [1]:
!hdfs dfs -ls /labs/laba01/ml-100k

Found 23 items
-rw-r--r--   3 hdfs hdfs       6750 2022-01-06 18:46 /labs/laba01/ml-100k/README
-rw-r--r--   3 hdfs hdfs        716 2022-01-06 18:46 /labs/laba01/ml-100k/allbut.pl
-rw-r--r--   3 hdfs hdfs        643 2022-01-06 18:46 /labs/laba01/ml-100k/mku.sh
-rw-r--r--   3 hdfs hdfs    1979173 2022-01-06 18:46 /labs/laba01/ml-100k/u.data
-rw-r--r--   3 hdfs hdfs        202 2022-01-06 18:46 /labs/laba01/ml-100k/u.genre
-rw-r--r--   3 hdfs hdfs         36 2022-01-06 18:46 /labs/laba01/ml-100k/u.info
-rw-r--r--   3 hdfs hdfs     236344 2022-01-06 18:46 /labs/laba01/ml-100k/u.item
-rw-r--r--   3 hdfs hdfs        193 2022-01-06 18:46 /labs/laba01/ml-100k/u.occupation
-rw-r--r--   3 hdfs hdfs      22628 2022-01-06 18:46 /labs/laba01/ml-100k/u.user
-rw-r--r--   3 hdfs hdfs    1586544 2022-01-06 18:46 /labs/laba01/ml-100k/u1.base
-rw-r--r--   3 hdfs hdfs     392629 2022-01-06 18:46 /labs/laba01/ml-100k/u1.test
-rw-r--r--   3 hdfs hdfs    1583948 2022-01-06 18:46 /labs/laba01/ml-100k/u2.base


In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))


In [ ]:
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

In [5]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

In [6]:
conf.set('spark.app.name', 'KEM_1_laba')

In [7]:
sc = SparkContext(conf=conf)

In [8]:
sc

<SparkContext master=yarn appName=KEM_1_laba>

In [9]:
data = sc.textFile('/labs/laba01/ml-100k/u.data')

In [12]:
data.take(10)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596',
 '298\t474\t4\t884182806',
 '115\t265\t2\t881171488',
 '253\t465\t5\t891628467',
 '305\t451\t3\t886324817',
 '6\t86\t3\t883603013']

In [13]:
data_split = data.map(lambda x: x.split('\t'))

In [71]:
data_split.take(10)

[['196', '242', '3', '881250949'],
 ['186', '302', '3', '891717742'],
 ['22', '377', '1', '878887116'],
 ['244', '51', '2', '880606923'],
 ['166', '346', '1', '886397596'],
 ['298', '474', '4', '884182806'],
 ['115', '265', '2', '881171488'],
 ['253', '465', '5', '891628467'],
 ['305', '451', '3', '886324817'],
 ['6', '86', '3', '883603013']]

In [17]:
my_film = data_split.filter(lambda x: x[1] == '96')

In [19]:
my_film.count()

295

In [23]:
rating = my_film.flatMap(lambda x: x[2])

In [24]:
rating.take(10)

['5', '1', '5', '5', '4', '5', '5', '4', '5', '4']

In [64]:
group = rating.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x+y)

In [65]:
group.take(10)

[('1', 6), ('4', 123), ('5', 103), ('3', 43), ('2', 20)]

In [66]:
res = group.sortBy(lambda x: x[0])

In [68]:
res.take(10)

[('1', 6), ('2', 20), ('3', 43), ('4', 123), ('5', 103)]

In [69]:
res2 = res.map(lambda x: x[1])

In [70]:
res2.take(10)

[6, 20, 43, 123, 103]

In [80]:
all_films = data_split.flatMap(lambda x: x[2])\
                    .map(lambda x: (x, 1))\
                    .reduceByKey(lambda x, y: x+y)\
                    .sortBy(lambda x: x[0])\
                    .map(lambda x: x[1])

In [81]:
all_films.take(10)

[6110, 11370, 27145, 34174, 21201]

In [82]:
import json

In [86]:
otvet = {
   "hist_film": res2.collect(),
   "hist_all": all_films.collect()
}

In [103]:
with open('lab01.json', 'w') as file:
    json.dump(otvet, file)

In [ ]:
!hdfs dfs -ls 


In [ ]:
sc.stop()